In [ ]:
! git clone https://github.com/Duchanoy/DeepDamper.git

In [ ]:
%cd DeepDamper

In [ ]:
!git config --global user.email "duch97@hotmail.com"
!git config --global user.name "Duchanoy"
!git remote rm origin
!git remote add origin https://token@github.com/Duchanoy/DeepDamper.git

In [ ]:
%tensorflow_version 2.x

In [ ]:
import json
import matplotlib.pyplot as plt
import numpy as np
import tensorflow as tf
import time
import os

In [ ]:
def read_json(name):
    #print(name)
    with open(name) as data_file:    
            data = json.load(data_file)
    return data

In [ ]:
def data_proces(name,Deface=100,NumSam=200):
    datos=read_json(name)
    entradas=read_json('_'.join(('inp',name)))
    train=np.array([[0,0,0,0,0,0,0,0,0],[0,0,0,0,0,0,0,0,0]])
    referencia=[0]
    for key in datos.keys():
        #print(len(datos[key]))
        for k,exp in enumerate(datos[key]):
            VelC=exp["velocity"]
            forceC=exp["force"]
            VelCj=[round(elem, 1) for elem in VelC]
            Conj=set(VelCj)
            infbar=100
            for elem in list(Conj):
                indices=[i for i,x in enumerate(VelCj) if x==elem]
                for ind in range(len(indices)-1):
                    if forceC[indices[ind]] == forceC[indices[ind+1]] and indices[ind]> 120:
                        reference = [indices[ind],indices[ind+1]]
                        break
                    elif abs(forceC[indices[ind]]-forceC[indices[ind+1]])<infbar and indices[ind+1]-indices[ind]>20 and indices[ind+1]-indices[ind]<200 and indices[ind]> 120:
                        tempref = [indices[ind],indices[ind+1]]
                        infbar = abs(forceC[indices[ind]]-forceC[indices[ind+1]])
                if 'reference' in locals():
                    break
            
            if 'reference' in locals():    
                Deface = reference[0]
                NumSam = reference[1] - reference[0]
                del reference
            else:
                Deface = tempref[0]
                NumSam = tempref[1] - tempref[0]
                del tempref
            referencia.append(referencia[-1]+NumSam+1)
            for element in range(NumSam+1):
                caso=list()
                caso.append(entradas[key]["Ap"])
                caso.append(entradas[key]["Ag"])
                caso.append(entradas[key]["h"])
                caso.append(entradas[key]["L"])
                caso.append(entradas[key]["viscocidad"])
                caso.append(entradas[key]["Ty"][k]/1000)
                caso.append((exp["velocity"][element+Deface])*0.001)
                caso.append(exp["displace"][element+Deface])
                caso.append(exp["force"][element+Deface])
                casonp=np.array(caso)
                #print(len(casonp))
                train=np.append(train,[casonp],axis=0)
            
    return train[2:][:],referencia


In [ ]:
def Target_C(data):
    Arn=(data[0]+((data[1])/2))
    Fn=((12*(data[4])*Arn*(data[6]/10))/((data[1])*data[2]*data[2]))
    Fn=Fn*(data[0]*data[3])
    Fty=data[8]-Fn
    c=((Fty*data[2])/(data[3]*data[0]*data[5]*1000))
    return Fty


In [ ]:
def force_limit(data):
    Arn=(data[0]+((data[1])/2))
    Fn=((12*(data[4])*Arn*(data[6]))/((data[1])*data[2]*data[2]))
    Fn=Fn*(data[0]*data[3])
    #c=2.07+((12*data[4]*data[0]*data[6])/((12*data[4]*data[0]*data[6])+(0.4*data[1]*data[2]*data[5])))
    c=1.07
    Fty= (c*data[3]*data[0]*data[5]*1000)/(data[2])
    fr=Fn+Fty
    #print(fr)
    #print(data[8])
    return fr

In [ ]:
train,refer=data_proces("twofluidexp.json",Deface=120,NumSam=260)
#rint(train.shape)
#print(train[1:4])
np.random.shuffle(train)
#train[1:4,8:9].shape
#print(train[20][])
#print(refer)

In [ ]:
Train=np.array(train)[:,:8].astype("float32")
Target=np.array(train)[:,8:9].astype("float32")

In [ ]:
checkpoint_path = "training_1/cp.ckpt"
checkpoint_dir = os.path.dirname(checkpoint_path)
# Create a callback that saves the model's weights
cp_callback = tf.keras.callbacks.ModelCheckpoint(filepath=checkpoint_path,
                                                 save_weights_only=True,
                                                 verbose=1)

## Arquitectura de la red usando el modelo de Keras por medio objetos

In [ ]:
class DeepDamper(tf.keras.Model):
  def __init__(self):
    super(DeepDamper, self).__init__()
    self.HL1 = tf.keras.layers.Dense(15, input_shape=(8,) , activation='relu')
    self.HL2 = tf.keras.layers.Dense(15, activation = 'relu')
    self.HL3 = tf.keras.layers.Dense(15, activation = 'relu')
    self.out = tf.keras.layers.Dense(1)
    # Seleccionar una funcion de error, optimizador y metricas de evaluacion 
    self.compile(optimizer=tf.keras.optimizers.Adam(0.0001), loss=tf.keras.losses.MeanSquaredError())

  def call(self, inputs):
    x = self.HL1(inputs)
    x = self.HL2(x)
    x = self.HL3(x)
    x = self.out(x)
    return x

In [ ]:
DamperModel = DeepDamper()
DamperModel.fit(x=Train, y=Target,batch_size =20, epochs=10000,callbacks=[cp_callback])

In [ ]:
DamperModel.save('my_net') 

In [ ]:
!git add -A
!git commit -m "Se agrego entrenamiento1"
!git push -u origin master